# Dynamic Programming Tutorial with Example Problem

In [1]:
from time import time
from operator import itemgetter
import numpy as np

## Introduction

Many scheduling problems are NP-hard, meaning among other things that the number of possible solutions grows exponentially with problem size. Searching efficiently through the solution space becomes very important for finding a high-quality solution in a reasonable amount of time, because otherwise we would be overwhelmed with candidate solutions and spend an inordinate amount of time collecting them all when we only need the best one.

Some problems exhibit what is called [optimal substructure](https://en.wikipedia.org/wiki/Optimal_substructure), meaning that the accumulation of greedy solutions of subproblems leads to the optimal solution of the full problem. The approach to solving problems through this lens is called [dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming).

This tutorial is based on a problem statement assigned to me during the hiring process at a certain company. I thought it makes a neat, encapsulated tutorial subject and put together this notebook to demonstrate the thought process of breaking down such a problem.

## What is dynamic programming?

I find that there are many still wrapping their heads around what exactly is dynamic programming. Some think it's just memoization plus recursion -- but really, it's more than that. It's an approach that exploits properties of the problem that allow for an elegant method to finding the optimal solution.

The approach is a bit like induction: first we solve a base case, then we show that the solution method gives an optimal solution for a problem of size $n$, then we show that the solution also works for a problem of size $n+1$ using one more step of the solution method. 

However, the solution method returns not just one solution but all viable (i.e., not discarded) solutions. That's okay because dynamic programming requires collecting high-quality solutions while discarding as many low-quality ones as early as possible. It's only at the end that we sort through them to find the solution that is optimal for our problem. Dynamic programming is thus the happiest marriage of induction, recursion, and greedy optimization.

The "dynamic" part of this approach is that we only have to apply one function repeatedly to the problem, and that function "dynamically" partitions the solution space into valid and invalid solutions as it progresses based on what it's already seen and where it is in its search.

## The Problem

### Problem Statement

You are an operations specialist in a company whose supply chain is undergoing restructuring. The machines you own are failing, so you've done your research and have collected a number of offers for buying and selling machines while your supply chain is brought back up to speed. 

* You may only use one machine at a time, and must sell the one you own before buying another. 

* You can only buy machines with the cash you have in hand at that instant. 

* Each machine $i$ has

  * a day of availability $D_i$, 
  
  * purchase cost $P_i$, 
  
  * resale value $R_i<P_i$, and 
  
  * a certain amount of profit generated per day $G_i$.

* You can't earn any money on a day where you are buying and selling machines since they must be taken offline first. 

Your task is to determine exactly which machines to buy and sell to maximize profit during the restructuring period.

Let's start by making a couple utilities that let us instantiate a problem case.

In [2]:
class Case:
    def __init__(self, inits, specs):
        self.inits = list(inits)
        self.orig_specs = specs
        self.specs = np.array(sorted(specs, key=itemgetter(0)))
    def __repr__(self):
        return self.inits.__repr__() + "\n" + self.orig_specs.__repr__()
        
        
def generate_case(N, C, D):
    specs = np.zeros((N, 4))
    specs[:,3] = np.random.randint(1, 2*np.sqrt(C), size=N)                    # Gi
    specs[:,0] = np.random.randint(1, D+1, size=N)                             # Di
    specs[:,2] = np.random.randint(1, C, size=N)                               # Ri
    specs[:,1] = np.array([np.random.randint(Ri+1, C+1) for Ri in specs[:,2]]) # Pi > Ri
    return Case((N,C,D), specs)

In [3]:
cases = [generate_case(6, 10, 20)]
cases[0]

[6, 10, 20]
array([[ 5.,  9.,  1.,  3.],
       [18., 10.,  9.,  5.],
       [ 6., 10.,  6.,  1.],
       [11.,  5.,  3.,  2.],
       [18.,  9.,  4.,  1.],
       [ 9., 10.,  9.,  1.]])

### How do we think about this problem?

To figure out how to attack this problem, let's start off by listing some of its properties:

1. for each machine on offer, there are two possible courses of action: either we buy the machine, or we do not;

2. the outcome of a given decision is not constrained by any previous or future decisions besides the constraints on the evolving state of the system;

3. if the cash-only constraint is violated at any time, any solutions derived from the present partial solution are infeasible.

Take some time to fully comprehend these properties and their implications. These are important for moving forward confidently in our decision to apply dynamic programming to this problem.

Property 1 indicates that the solution space is of size $2^N$. This is because, at each step in the solution, our solution space doubles as there are two possible paths from that step. Thus the solution space grows exponentially with increasing $N$, and we are dealing with a case where our strategy for exploring the solution space requires some consideration.

Property 2 indicates that this problem is decomposable into subproblems with identical properties as the original. To get a sense of this, consider: after step $m < N$ in constructing a solution, we can treat the current partial solution as a full solution to the subproblem where only $m$ machines are provided in the case. We can then solve the problem of size $m+1$, $m+2$, and so on up to $N$. We are confident of the optimality of the final solution because the solution of each subproblem along the way was also optimal, and each step used the same method to go from one level to the next.

Property 3 is the key to efficiently searching the solution space. As a corrolary to property 2, we know also that any condition that prevents a solution of a given subproblem from being feasible also disqualifies all solutions which are derived from the subproblem. We will exploit this later in two different ways to speed up our algorithm considerably.

### Modelling the Problem: Recursion and Trees

So how do we model our progress through the problem? We want a data structure which exhibits the same properties as the problem. In particular, we want a data structure which contains smaller versions of itself (self-similarity) and which models efficiently a series of steps with a finite and integer number of choices at each step (branching process). How we traverse the data structure should also reflect the problem: we require that there is a unique path through the steps that arrives at a unique solution. Finally, because we want to be agnostic to the number of steps taken so far or remaining, we must build an algorithm which applies the same function at each step in the process.

Putting all this together, we recognize that *recursion* will play a large role. Recursion is nothing more than applying the same function to a smaller version of the original input until we reach a "base case", which is triggered when the traversal reaches a full solution.

In any case, we now expect that to solve this problem, we need to find all feasible solutions by traversing the tree, then simply compare the full set and return the one(s) that achieve(s) the highest score.

Our last insight to an efficient solution is the conclusion drawn from property 3 of the problem listed above. We can see that if the progress of building a solution leads to a violation of constraints, the entire subtree will also be invalid because any state of the system that is achieved during any further progress toward a solution is impossible according to the definition of the problem. Then we can abandon any further evaluation of that subtree and instead search for solutions that we know do not violate the constraints.

Our base case is the state where we have made decisions about all machines and are ready to return our final cash amount.

If we only have a partial solution, we take the next step by making a decision about the next machine and then paying out and collecting money as defined by the problem statement. We carry this money with us through the evaluations to keep track of how much we earned. If at any point the cash-only constraint is violated, the function neglects to search further and returns an empty list prematurely. We design this behavior into the function to avoid performing calculations on what we can prove are futile tasks, and instead spend the time and effort searching among solutions we know to be feasible.

I think this last point is worth spending some time on, because it's at the crux of confusion for many people. The "memoization" that many people hear about is nothing other than storing the solutions that are explored fully and checking them at the end. Rather, the important storage of information is the state of the system as it passes through and is modified by the steps that lead to a solution. The flip side of the memoization coin is that we throw away the solutions that we *don't* want in addition to keeping the solutions we do.

### Implementing a Solution

Below we find an instance of a recursive function which solves this scheduling problem via dynamic programming. Comments serve as annotation to describe the different components of the function.

In [4]:
class Machine:
    def __init__(self, p, r, g):
        self.p = p
        self.r = r
        self.g = g
    def __repr__(self):
        return "[" + ", ".join(map(str, [self.p, self.r, self.g])) + "]"

    
def dp_recurse(state_vector, specs, solution=[]):
    
    """
    A recursive function which solves the scheduling problem via dynamic programming. 
    This function constructs a set of solutions together with their scores by successively solving 
    larger and larger subproblems of the main problem. If at any point a constraint is violated,
    the function returns prematurely and no solutions are collected from that branch of the tree.
    """
    
    # We carry around a state vector to keep track of important values like 
    # how many days have passed, the machine's attributes, and the money we earned.
    # These are updated in every step and passed down through the tree.
    day, machine, cash, last_day = state_vector
    
    if len(specs) == 0:
        
        # The base case. When there are no machines left in specs (a decision has been made about each machine),
        # we have reached a full solution and can return the total money.
        
        # Add the rest of the profit plus the money we get from selling the machine.
        cash += machine.g * (last_day - day) + machine.r
        
        # We return a list containing a tuple of score and solution. 
        # This list is appended to those that are returned from all the other leaf nodes, 
        # and these together form the full set of feasible solutions.
        soln_str = "".join(map(str, solution))
        return [(cash, soln_str)]
    
    else:
        
        # If we reach this point, we have not obtained a full solution 
        # and must continue through the tree toward a leaf node.
        
        # We need to collect and organize values which are important for the next step we are about to take.
        new_day, new_p, new_r, new_g = specs[0]
        new_machine = Machine(new_p, new_r, new_g)
        
        
        if new_day == day:  
            
            # We need this to account for multiple machines offered on the same day.
            # You may notice that it's possible to buy and sell multiple machines per day with this setup.
            # However, we don't need to worry about these cases because they will always be suboptimal and thus ultimately discarded.
            # To see this, consider: buying and selling the same machine on the same day means 
            # we were never able to start the machine and generate profit with it.
            # That and R < P for each machine means we always experience a net loss 
            # if we buy two or more machines per day.
            
            new_cash = cash
            new_cash_zero = new_cash
        
        else:
            
            # Otherwise, some time has passed and the machine has generated profit for us.
            # Note: below, we are keeping track of two different money amounts,
            # corresponding to the two outcomes of a decision to buy a machine.
            
            # This is the amount of cash we accumulated before "today" started, 
            # i.e., the amount of profit our machine made up until the morning of `new_day`.
            new_cash = cash + machine.g * (new_day - day - 1)
            
            # If we don't buy a machine, the one we have stays running and making us money.
            new_cash_zero = new_cash + machine.g
        
        # If we do buy a machine, we gain the resale value of the old machine 
        # and lose the purchase cost of the new one.
        new_cash_one = new_cash + machine.r - new_machine.p
        
        solutions = []
        
        if new_cash_zero >= 0:
            
            # For this outcome, our state changes in that:
            # time has elapsed,
            # the machine from the last step is still in our possession,
            # and we have made some money from our operations.
            
            new_state_zero = [new_day, machine, new_cash_zero, last_day]
            
            # This is the heart of the recursion: 
            # we call the same function again on an updated system 
            # and solve a slightly larger (or smaller, depending on your perspective) 
            # problem than the one in the previous level.
            # The function call returns a list containing all solutions
            # in the related paths down the tree, so we collect that list
            # into a single object and pass it up to the next level. Once it gets
            # to the top (root) of the tree, we have collected every solution from every branch.
            # With `specs`, we pass along to the next level only the machines that have not yet been considered;
            # the other ones have already been fully integrated into the solution so far,
            # so we don't have to carry them around anymore.
            # This has an additional benefit: when the specs are all gone, we know we have
            # completely defined a solution and the base case is reached.
            solutions += dp_recurse(new_state_zero, specs[1:], solution+[0])
        
        if new_cash_one >= 0:
            
            # This condition will only trigger if our constraint has not been violated.
            # That is, if we have to go into debt to purchase a machine, this branch
            # does not lead to any feasible solutions and we ignore it.
            
            new_state_one = [new_day, new_machine, new_cash_one, last_day]
            solutions += dp_recurse(new_state_one, specs[1:], solution+[1])
            
        # Once we've collected the solutions from both subtrees, 
        # we send them up toward the top of the tree where they are used to find the best solution.
        return solutions

### Branch and Bound

We can apply additional constraints, but only if we are capable of proving that solutions meet or violate those constraints. For the case of the cash constraint, we simply neglect searching further beyond an action which results in a net cash amount less than zero. We can neglect also solution paths which are provably incapable of reaching a score above the best among all solutions found so far. This is called [branch and bound](https://en.wikipedia.org/wiki/Branch_and_bound) and is the basis for many efficient combinatorial search techniques, most notably in powerful (hard-coded) chess engines today as *alpha-beta pruning*.

For our case, we can define a crude upper bound of the possible solution paths' scores as the sum of: the cash in hand today, the amount of profit made if the best remaining machine started today, and the maximum resale value among all remaining machines. This is the simplest form of upper bound, albeit inaccurate except in rare circumstances. Nonetheless, we know that each solution will respect this upper bound because we extrapolate using the absolute fastest rate of cash inflow. Adding the maximum resale value also accounts for any gains we may have made from buying and selling machines.

In [5]:
lower_bound = 0

def dp_recurse_bnb(state_vector, specs, solution=[]):
    
    """
    A recursive function which uses dynamic programming 
    and an additional branch-and-bound constraint 
    to further restrict the search space to feasible optima.
    """
    
    # Addition one: a global variable representing the best score achieved in the search so far.
    global lower_bound
    
    day, machine, cash, last_day = state_vector
    
    if len(specs) == 0:
        cash += machine.g * (last_day - day) + machine.r
        
        # Addition two: a way to update the lower bound when a new, better solution is found.
        if lower_bound < cash:
            lower_bound = cash
            
        soln_str = "".join(map(str, solution))
        return [(cash, soln_str)]
    
    else:
        profit_w_best = max(machine.g, max(specs[:,3])) * (last_day - day)
        resale_w_best = max(machine.r, max(specs[:,2]))
    
    # Addition three: a way to abort further searching in the subtree 
    # when the partial solution is proven to be infeasible.
    # Here, an upper bound is calculated for the money this partial solution may eventually earn. 
    # If the upper bound of the solutions derived from here is still lower than the current best solution, 
    # there's no point in continuing.
    # Calculated as sum of: current money, profit if best remaining machine started today, 
    # and the maximum resale value out of the remaining machines. Although lenient, we will
    # see it is still surprisingly effective.
    if cash + profit_w_best + resale_w_best <= lower_bound:
        return []
    
    else:
        new_day, new_p, new_r, new_g = specs[0]
        new_machine = Machine(new_p, new_r, new_g)
        if new_day == day:
            new_cash = cash
            new_cash_zero = new_cash
        else:
            new_cash = cash + machine.g * (new_day - day - 1)
            new_cash_zero = new_cash + machine.g
        new_cash_one = new_cash + machine.r - new_machine.p
        solutions = []
        if new_cash_zero >= 0:
            new_state_zero = [new_day, machine, new_cash_zero, last_day]
            solutions += dp_recurse_bnb(new_state_zero, specs[1:], solution+[0])
        if new_cash_one >= 0:
            new_state_one = [new_day, new_machine, new_cash_one, last_day]
            solutions += dp_recurse_bnb(new_state_one, specs[1:], solution+[1])
        return solutions

Now we slap together all the logic and create a singular function which solves every case in a list of cases using the recursive function of your choosing.

In [6]:
def solve(f_recurse, cases, verbose=False):
    
    """
    Solve all "cases" using preferred recursive function.
    """
    
    global lower_bound
    
    output_str = ""
    
    for k,case in enumerate(cases):
        
        lower_bound = 0        
        state_vector = [0, Machine(0,0,0)] + case.inits[1:]  # we start at day 0 with effectively no machine
        
        # This single call constructs a list of solutions from a tree of solution paths 
        # by recursing until every node in the tree is either 
        # safely discarded or explored to the leaves.
        solutions = f_recurse(state_vector, case.specs)
        
        # Here we use a trick where sorting a list of tuples only sorts by the first element of each tuple. 
        # So we sort by money made, descending, to get the best solutions.
        srt_solns = sorted(solutions, reverse=True)
        
        # The best solution is the one with the most money after the period has elapsed.
        best_profit, best_solution = srt_solns[0]  # note: best_solution returned in order of specs stored in Case instance, not necessarily the order given in the file
        case_str = "Case {}: {:d}".format(k+1, int(best_profit))
        if verbose:
            print(case_str)
        
        # Assemble the output in a descriptive fashion.
        output_str += case_str + "\n"
        
    with open("output.txt", 'w') as f:
        
        # Put the maximum score for each case in a file and save it to disk.
        f.write(output_str)

And test everything:

In [7]:
solve(dp_recurse, cases, True)
solve(dp_recurse_bnb, cases, True)

Case 1: 47
Case 1: 47


### Testing Convergence Speed

In [8]:
gen_cases = [generate_case(20, 20, 50) for _ in range(100)]

Let's see how much branch and bound helps our search. We will run each recursive function on each generated case and get a sense of the time difference between them.

In [9]:
t0 = time()
solve(dp_recurse, gen_cases)
t1 = time()
solve(dp_recurse_bnb, gen_cases)
t2 = time()
vanilla_time = t1-t0
bnb_time = t2-t1
print("Time for vanilla DP: {:.1f}s for {:d} cases".format(vanilla_time, len(gen_cases)))
print("Time for   BnB   DP: {:.1f}s for {:d} cases".format(bnb_time, len(gen_cases)))
if vanilla_time > bnb_time:
    print("Vanilla is {:.1f}x slower".format(vanilla_time/bnb_time - 1))
else:
    print("  BnB   is {:.1f}x slower".format(bnb_time/vanilla_time - 1))

Time for vanilla DP: 1348.1s for 100 cases
Time for   BnB   DP: 3.9s for 100 cases
Vanilla is 340.7x slower


So branch and bound speeds up the wall-clock solution time considerably by pruning the tree's branches that are useless to continue evaluating.

In general, branch and bound will provide quicker solutions compared to the vanilla implementation when the problem size is larger (dependent on how you define the bounding heuristic). There are two interrelated reasons for this: discarding partial solutions high up in the tree removes a large number of node evaluations, and this effect is much larger when the tree is much deeper and the pruning happens sooner. Discarding one subtree at level $m$ of the whole tree removes $2^{n-m}$ solutions from consideration. If $m$ is small, the solution space is cut down by a relatively large fraction and convergence occurs faster by virtue of requiring the exploration of fewer solutions.

## Conclusion

In this tutorial you learned what dynamic programming really is, and how to properly approach a problem that warrants it. You also learned that dynamic programming is not incompatible with other methods and indeed benefits from ones which discard low-quality solutions early.

There are many other ways to solve this problem, for instance brute force search or evolutionary algorithms, but none as thorough, direct, and efficient.